In [1]:
from dotenv import load_dotenv
load_dotenv()
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import (
    TimeoutException, 
    NoSuchElementException, 
    StaleElementReferenceException,
    ElementClickInterceptedException,
)
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import datetime
import pickle, json, time, os
import regex as re
import pandas as pd

In [2]:
def gen_searches():
    
    sources = ['rst=sfusat', 'rst=sfwsj', 'rst=sfnyt', 'rst=sfglob', '(rst=ftft or rst=ftcom)']
    companies = ['applc']

    return [f'{source} and fds={company}' for source in sources for company in companies]

In [3]:
def enter_search(driver, wait, dates, year, search):
    
    attempts = 0
    while attempts < 5:
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, '//select[@name="dr"]'))).click()
            wait.until(EC.element_to_be_clickable((By.XPATH, '//option[@value="Custom"]'))).click()
            driver.find_element(By.ID, 'frm').clear()
            driver.find_element(By.ID, 'frm').send_keys('01')
            driver.find_element(By.ID, 'frd').clear()
            driver.find_element(By.ID, 'frd').send_keys('01')
            driver.find_element(By.ID, 'fry').clear()
            driver.find_element(By.ID, 'fry').send_keys(year)
            driver.find_element(By.ID, 'tom').clear()
            driver.find_element(By.ID, 'tom').send_keys(dates[0])
            driver.find_element(By.ID, 'tod').clear()
            driver.find_element(By.ID, 'tod').send_keys(dates[1])
            driver.find_element(By.ID, 'toy').clear()
            driver.find_element(By.ID, 'toy').send_keys(year)
            driver.find_element(By.XPATH, '//select[@name="isrd"]').click()
            wait.until(EC.element_to_be_clickable((By.XPATH, '//option[@value="High"]'))).click()
            search_box = driver.find_element(By.XPATH, '//textarea[@name="ftx"]')
            search_box.clear()
            search_box.send_keys(search)
            try:
                driver.find_element(By.XPATH, '//div[@class="pillNoMenu"]').click()
            except NoSuchElementException:
                pass
            search_box.send_keys(Keys.ENTER)
            wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@data-channel="Dowjones"]')))
            time.sleep(5)
            break
        except UnexpectedAlertPresentException:
            attempts += 1

In [4]:
def next_loop(driver, wait):
    
    attempts = 0
    while attempts < 5:
        try:
            wait.until(EC.visibility_of_element_located((By.XPATH, '//a[@class="nextItem"]'))).click()
            wait.until(EC.visibility_of_element_located((By.XPATH, '//img[@src="../img/listmanager/progress.gif"]')))
            wait.until(EC.invisibility_of_element_located((By.XPATH, '//img[@src="../img/listmanager/progress.gif"]')))
            break
        except (StaleElementReferenceException, ElementClickInterceptedException):
            attempts += 1

In [6]:
def get_page_info(soup, counter):
    
    article_links = []
    map_dict = {
        'January': '1',
        'February': '1',
        'March': '1',
        'April': '2',
        'May': '2',
        'June': '2',
        'July': '3',
        'August': '3',
        'September': '3',
        'October': '4',
        'November': '4',
        'December': '4',
    }
    
    for headline in soup.find_all('tr', {'class': 'headline'}):
            
        article_links.append(headline.find('a').get('href'))
        sub_list = sum([subtitle.split(' ') for subtitle in headline.find('div').text.split(',') if len(re.findall('[0-9]+', subtitle)) > 0], [])
        month = ''.join(set(sub_list) & set(map_dict.keys()))
        counter[map_dict[month]] += 1
        
    return counter, article_links

In [9]:
def get_all_freq(eid_username, eid_password, search_function, first_login):
    
    """
    Argument explanations:
    
    1. eid_username
    
        - Should be your UT EID username that you use to login to Canvas
    
    2. eid_password
    
        - Should be your password to login to Canvas
        - I recommend you putting the password in a .env file and using the following format for the password argument:
        os.getenv("password_variable_name") where password variable name is a name you choose to define your password variable
        inside of the .env folder.
        
    3. search_function
        
        - Is defined above by inputting a dataframe filled with company codes
        
    4. first_login
    
        - A boolean that is True if your browser has not yet logged into Factiva and False otherwise. Every time you
        reopen the browser, you need to set this to True because closing the browser deletes some of your session cookies. Afterwards, you
        may return to the main search page by setting this value to false and rerunning the function.
        
    """
    
    url = 'https://guides.lib.utexas.edu/db/144'
    option = webdriver.ChromeOptions()
    option.add_experimental_option("debuggerAddress", "localhost:9222")
    driver = webdriver.Chrome(executable_path = "C://Users//galon//cd_secure//chromedriver.exe",options=option)
    wait = WebDriverWait(driver, 10)
    driver.set_page_load_timeout(20)
    driver.get(url)
    
    if first_login == True:

        wait.until(EC.element_to_be_clickable((By.ID, 'username')))
        driver.find_element(By.ID, 'username').send_keys(eid_username)
        driver.find_element(By.XPATH, '//input[@id="password"]').send_keys(eid_password)    
        driver.find_element(By.XPATH, "//input[@value='Sign in']").click()
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@id='trust-browser-button']"))).click()
            
        except TimeoutException:
            print('Duo cookies still valid; proceeding to search page...')   
    print(f'\nStarting at: {datetime.now()}\n')
    searches = search_function
    df = pd.DataFrame.from_dict({'year': [], 'quarter': [], 'count': []})
    
    for search in searches:
    
        pub_code = search.split('and')[0].strip()
        co_code = search.split('fds=')[-1]
        
        for year in range(1995, 2021):
            duplicates = 0
            counter = {'1': 0, '2': 0, '3': 0, '4': 0}            
            if year < 2020:
                dates = ('12', '31')
            else:
                dates = ('06', '30')
            enter_search(driver, wait, dates, year, search)
            
            while year:
                
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                if soup.find('tr', {'class': 'headline'}) == None:
                    wait.until(EC.element_to_be_clickable((By.ID, 'btnModifySearch'))).click()
                    print(f'year: {year}\nNo search results')
                    break                    
                elif soup.find('a', {'class', 'nextItem'}) !=  None:
                    counter, article_links = get_page_info(soup, counter)
                    duplicates += int(soup.find('span', {'id': 'dedupSummary'}).text.split(':')[1].strip())
                    next_loop(driver, wait)
                elif soup.find('a', {'class', 'nextItem'}) ==  None:
                    duplicates += int(soup.find('span', {'id': 'dedupSummary'}).text.split(':')[1].strip())
                    results_bar = soup.find('span', {'class': 'resultsBar'}).text.split()
                    total = re.sub(',','',str(results_bar[3]))
                    ttotal = re.sub(',','',str(results_bar[5]))
                    if (int(ttotal)-duplicates) != int(total):
                        return 'Did not count duplicates properly; adjust sleep time'
                    else:
                        print(f'year: {year}\nDuplicates are equal')
                        wait.until(EC.element_to_be_clickable((By.ID, 'btnModifySearch'))).click()
                        break
                        
            df = pd.concat([df, pd.DataFrame.from_dict(
                {
                        'year': [year, year, year, year],
                        'quarter': list(counter.keys()),
                        'count': list(counter.values()),
                        'company_code': [co_code, co_code, co_code, co_code],
                        'pub_code': [pub_code,pub_code,pub_code,pub_code,]
                }
            )])
    print(f'Done!\n{datetime.now()}')
    return df

In [11]:
get_all_freq('gal767', os.getenv('eid_password'), gen_searches(), first_login=False)

C:\Users\galon\AppData\Local\Temp\ipykernel_8596\587917110.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "C://Users//galon//cd_secure//chromedriver.exe",options=option)



Starting at: 2022-10-27 01:25:12.863510

year: 1995
No search results
year: 1996
Duplicates are equal
year: 1997
Duplicates are equal
year: 1998
Duplicates are equal
year: 1999
No search results
year: 2000
Duplicates are equal
year: 2001
Duplicates are equal
year: 2002
Duplicates are equal
year: 2003
Duplicates are equal
year: 2004
Duplicates are equal
year: 2005
Duplicates are equal
year: 2006
Duplicates are equal
year: 2007
Duplicates are equal
year: 2008
Duplicates are equal
year: 2009
Duplicates are equal
year: 2010
Duplicates are equal
year: 2011
Duplicates are equal
year: 2012
Duplicates are equal
year: 2013
Duplicates are equal
year: 2014
Duplicates are equal
year: 2015
Duplicates are equal
year: 2016
Duplicates are equal
year: 2017
Duplicates are equal
year: 2018
Duplicates are equal
year: 2019
Duplicates are equal
year: 2020
Duplicates are equal
year: 1995
No search results
year: 1996
No search results
year: 1997
Duplicates are equal
year: 1998
Duplicates are equal
year: 1999

ElementClickInterceptedException: Message: element click intercepted: Element <li id="btnModifySearch" class="btn" onclick="">...</li> is not clickable at point (672, 124). Other element would receive the click: <div id="__overlayBackground" style="position: absolute; top: 0px; left: 0px; display: block; z-index: 102100; background: rgb(255, 255, 255); opacity: 0; height: 1334px; width: 751px;"></div>
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x01032733+2434867]
	Ordinal0 [0x00FC07A1+1968033]
	Ordinal0 [0x00EAC678+837240]
	Ordinal0 [0x00EDFC07+1047559]
	Ordinal0 [0x00EDDBE8+1039336]
	Ordinal0 [0x00EDB8DB+1030363]
	Ordinal0 [0x00EDA639+1025593]
	Ordinal0 [0x00ED0273+983667]
	Ordinal0 [0x00EF443C+1131580]
	Ordinal0 [0x00ECFC54+982100]
	Ordinal0 [0x00EF4654+1132116]
	Ordinal0 [0x00F04892+1198226]
	Ordinal0 [0x00EF4256+1131094]
	Ordinal0 [0x00ECE840+976960]
	Ordinal0 [0x00ECF736+980790]
	GetHandleVerifier [0x012A3C72+2515426]
	GetHandleVerifier [0x0129702F+24